Reading in the data and then filtering out SNPs which do not meet p value significance before joining the data together.

In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl

# exp: Exposure
# out: Outcome
# ea: Exposure allele
# oa: Other allele

exp_header_dict = {
  'rsID':'rsid',
  'CHROM':'chr_exp',
  'ALT':'ea_exp',
  'REF':'oa_exp',
  'POOLED_ALT_AF':'eaf_exp',
  'EFFECT_SIZE':'beta_exp',
  'SE':'se_exp',
  'pvalue':'pval_exp'
}

out_header_dict = {
  'markername':'rsid',
  'chr':'chr_out',
  'bp_hg19':'pos_out',
  'effect_allele':'ea_out',
  'noneffect_allele':'oa_out',
  'effect_allele_freq':'eaf_out',
  'beta':'beta_out',
  'se_dgc':'se_out',
  'p_dgc':'pval_out'}

pthresh = 5e-8

# Renaming columns and filtering data to only include observations which fulfill significance threshold
dexp = (pl.scan_csv("dataset/ldlc_gwas.txt",separator="\t")
        .rename(exp_header_dict)
        .filter((pl.col('pval_exp') < pthresh)))
dout = (pl.scan_csv("dataset/mi_gwas.tsv",separator="\t")
        .rename(out_header_dict))

combined = (dexp.join(dout, on='rsid')
            # Convert all data to lowercase
            .with_columns(
                pl.col('ea_exp').str.to_lowercase(),
                pl.col('oa_exp').str.to_lowercase(),
                pl.col('ea_out').str.to_lowercase(),
                pl.col('oa_out').str.to_lowercase(),
            # Convert minor allele freq to effect allele freq
                pl.col('eaf_exp').mul(-1).add(1))
            .collect())

print(combined.shape)

(75089, 26)


We can use the provided harmonize function.

In [2]:
from MR.harmonize import harmonize

total = harmonize(combined, palindromic_action=1, palindromic_threshold=0.08)
print(total.shape)

# Seeing if duplicates are dropped
print(total.unique(subset=['rsid']).shape)

(73095, 26)
(73095, 26)


Clumping the data based on Linkage Disequilibirum (LD). This is to filter out SNPs which are closely correlated in order to essentially avoid double counting results into the final calculation.

In [3]:
from MR.ld import ld_clump

pruned_rsids = ld_clump(total['rsid'], total['pval_exp'])

processed_data = (total.join(pruned_rsids, on='rsid'))

print(processed_data.shape)

(379, 26)


In [4]:
from MR.ld import ld_matrix

# print(ld_matrix(processed_data['rsid']))

Calculating causal effects.

In [17]:
from MR.mr_methods.egger_regression import mr_egger_regression
from MR.mr_methods.inverse_variance_weighted import mr_inverse_variance_weighted
from MR.mr_methods.maximum_likelihood import mr_maximum_likelihood
from MR.mr_methods.median import mr_penalised_weighted_median, mr_simple_median, mr_weighted_median
from MR.mr_methods.mode import mr_simple_mode, mr_weighted_mode
from MR.mr_methods.outliers import mr_presso, mr_remove_outliers
from MR.mr_methods.wald_ratio import mr_wald_ratio


data = mr_remove_outliers(processed_data, 'cooks distance')
# data = processed_data

mr_presso(data, k=100)

# print('Inverse Variance Weighted')
# result = mr_inverse_variance_weighted(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Wald ratio')
# result = mr_wald_ratio(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Median')
# result = mr_simple_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'], 1000)
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Median')
# result = mr_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# # print('Penalised Weighted Median')
# # result = mr_penalised_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# # print(f'Effect: {result["effect"]}')
# # print(f'se: {result["se"]}')
# # print(f'pval: {result["pval"]}\n')

# print('Egger Regression')
# result = mr_egger_regression(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Mode')
# result = mr_simple_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Mode')
# result = mr_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# # print('Penalised Weighted Mode')
# # result = mr_penalised_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# # print(f'Effect: {result["effect"]}')
# # print(f'se: {result["se"]}')
# # print(f'pval: {result["pval"]}\n')

# print('Maximum Likelihood')
# result = mr_maximum_likelihood(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')


0.0
[0.0, 49.27, 219.82, 30.32, 121.28, 174.34, 26.53, 11.37, 341.1, 132.65, 26.53, 45.48, 7.58, 151.6, 83.38, 45.48, 325.94, 125.07, 348.68, 306.99, 333.52, 367.63, 30.32, 337.31, 269.09, 333.52, 30.32, 53.06, 0.0, 18.95, 288.04, 360.05, 45.48, 0.0, 261.51, 360.05, 121.28, 178.13, 0.0, 318.36, 242.56, 0.0, 231.19, 15.16, 306.99, 83.38, 348.68, 64.43, 140.23, 212.24, 234.98, 352.47, 375.21, 30.32, 140.23, 18.95, 83.38, 0.0, 60.64, 325.94, 90.96, 329.73, 11.37, 193.29, 375.21, 246.35, 7.58, 242.56, 75.8, 303.2, 0.0, 162.97, 318.36, 11.37, 155.39, 348.68, 11.37, 140.23, 136.44, 147.81, 314.57, 360.05, 0.0, 26.53, 174.34, 94.75, 18.95, 257.72, 98.54, 314.57, 204.66, 185.71, 231.19, 132.65, 0.0, 0.0, 0.0, 360.05, 49.27, 295.62, 22.74, 299.41, 3.79, 34.11, 68.22, 125.07, 0.0, 144.02, 15.16, 147.81, 151.6, 0.0, 379.0, 136.44, 7.58, 246.35, 371.42, 34.11, 322.15, 193.29, 272.88, 234.98, 144.02, 98.54, 37.9, 227.4, 0.0, 45.48, 216.03, 49.27, 162.97, 284.25, 193.29, 18.95, 306.99, 280.46, 310.7